In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import torch
import transformers
# import datasets
# import shap

# # load the emotion dataset
# dataset  = datasets.load_dataset("emotion", split = "train")
# data = pd.DataFrame({'text':dataset['text'],'emotion':dataset['label']})

c:\Users\asif.raza\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Data Facts and Import 

In [2]:
df_train = pd.read_csv('Dataset/SentimentAnalysisofTweetsthroughAltmetrics/train.csv')
df_test = pd.read_csv('Dataset/SentimentAnalysisofTweetsthroughAltmetrics/test.csv')

In [3]:
df_train.columns = ["Text", "Label"]
df_test.columns = ["Text", "Label"]

In [4]:
# remove the neutural.

#df_train= df_train[df_train['Is_Response'] != 0]
#df_test= df_test[df_test['Is_Response'] != 0]

In [4]:
df_train.shape

(732, 2)

In [5]:
df_test.shape

(314, 2)

In [6]:
df_train.head()

,Text,Label
0,good acronym copper nanotubes Definitely,-1
1,Author Michael Walz Wilhelm,0
2,GlycemicIndex diet restricted energy effective...,1
3,higher fibre intake partic cereal fibre reduce...,1
4,next life going research copper nanotubes CuNTs,-1


In [7]:
df_test.head()

,Text,Label
0,Yeah paper ebirdf,1
1,Nutrients Free Full Text,0
2,platform Bioinformatics paper advanced access ...,1
3,Regional distribution styrene analogues genera...,0
4,Duan naturally award Best Science Acronym year,-1


In [8]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    732 non-null    object
 1   Label   732 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 11.6+ KB


In [9]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314 entries, 0 to 313
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    314 non-null    object
 1   Label   314 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.0+ KB


In [10]:
df_train.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Label,732.0,0.019126,0.801616,-1.0,-1.0,0.0,1.0,1.0


In [11]:
df_test.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Label,314.0,0.009554,0.8013,-1.0,-1.0,0.0,1.0,1.0


### Data Cleaning / EDA

In [12]:
# ### Checking Missing values in the Data Set and printing the Percentage for Missing Values for Each Columns ###

# count = df_train.isnull().sum().sort_values(ascending=False)
# percentage = ((df_train.isnull().sum()/len(df_train)*100)).sort_values(ascending=False)
# missing_data = pd.concat([count, percentage], axis=1, keys=['Count','Percentage'])

# print('Count and percentage of missing values for the columns:')

# missing_data

In [13]:
# ### Checking for the Distribution of Default ###
# import matplotlib.pyplot as plt
# %matplotlib inline
# print('Percentage for default\n')
# print(round(df_train.Is_Response.value_counts(normalize=True)*100,2))
# round(df_train.Is_Response.value_counts(normalize=True)*100,2).plot(kind='bar')
# plt.title('Percentage Distributions by review type')
# plt.show()

In [14]:
#Removing columns
#df_train.drop(columns = ['User_ID', 'Browser_Used', 'Device_Used'], inplace = True)

In [16]:
# #This function converts to lower-case, removes square bracket, removes numbers and punctuation
# def text_clean_1(text):
#     text = text.lower()
#     text = re.sub('\[.*?\]', '', text)
#     text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
#     text = re.sub('\w*\d\w*', '', text)
#     return text

# cleaned1 = lambda x: text_clean_1(x)

In [15]:
# # Apply first level cleaning

# # Let's take a look at the updated text
# df_train['cleaned_description'] = pd.DataFrame(df_train.Description.apply(cleaned1))
# df_train.head(10)

In [17]:
# # Apply a second round of cleaning
# def text_clean_2(text):
#     text = re.sub('[‘’“”…]', '', text)
#     text = re.sub('\n', '', text)
#     return text

# cleaned2 = lambda x: text_clean_2(x)

In [18]:
# # Let's take a look at the updated text
# df_train['cleaned_description_new'] = pd.DataFrame(df_train['cleaned_description'].apply(cleaned2))
# df_train.head(10)

### spliting the data.

In [12]:
from sklearn.model_selection import train_test_split

# Independent_var = df_train.cleaned_description_new
# Dependent_var = df_train.Is_Response

# IV_train, IV_test, DV_train, DV_test = train_test_split(Independent_var, Dependent_var, test_size = 0.1, random_state = 225)

IV_train = df_train.Text
DV_train = df_train.Label
IV_test = df_test.Text
DV_test = df_test.Label


print('IV_train :', len(IV_train))
print('IV_test  :', len(IV_test))
print('DV_train :', len(DV_train))
print('DV_test  :', len(DV_test))


IV_train : 732
IV_test  : 314
DV_train : 732
DV_test  : 314


In [13]:
import torch
torch.cuda.is_available()

False

In [1]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

NameError: name 'torch' is not defined

In [19]:
torch.device('cpu')


device(type='cpu')

### Models 

In [21]:
# load the model and tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained("nateraw/bert-base-uncased-emotion", use_fast=True)
model = transformers.AutoModelForSequenceClassification.from_pretrained("nateraw/bert-base-uncased-emotion").cuda()

# build a pipeline object to do predictions
pred = transformers.pipeline("text-classification", model=model, tokenizer=tokenizer, device=0, return_all_scores=True)

AssertionError: Torch not compiled with CUDA enabled